In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import cufflinks as cf
import yfinance as yf
import talib
import scipy.stats as stats
import requests
import os
import win32com.client as win32
from datetime import datetime

Helper Functions

In [2]:
def get_historical_data(ticker):
    df = yf.download(tickers = ticker, auto_adjust=True)
    starting_period = df.index.min()
    ending_period = df.index.max()
    print("")
    print("The Historical data of {} stock  starts from {}".format(ticker,starting_period))
    print("The Historical data of {} stock ends at {}".format(ticker,ending_period))
    print("")
    return df

In [3]:
def get_fundamental_data(ticker):
    try: 
        
        all_data = yf.Ticker(ticker).info
        print("Key fundamental data about {} stock".format(ticker))
        print("==============================================")

        print("The Price-to-Earnings ratio of {} stock is : {}".format(ticker,all_data["trailingPE"]))
        print("The price-to-Book value of {} stock is : {}".format(ticker, all_data['priceToBook']))
        print("The Enterprise Value-to-EBITDA of {} stock is : {}".format(ticker, all_data['enterpriseToEbitda']))
    except KeyError as key_err:
            print(f"KeyError occurred: {key_err}")
    fundamental_data_df = pd.DataFrame(all_data)
    
    return fundamental_data_df

In [4]:
def get_sma(ticker,start_date,end_date):
    df = yf.download(tickers = ticker, start = start_date, end = end_date)
    sma_7 = 7
    sma_30 = 30
    sma_200 = 200
    df["7 days SMA"] = df.Close.rolling(sma_7).mean()
    df["30 days SMA"] = df.Close.rolling(sma_30).mean()
    df["200 days SMA"] = df.Close.rolling(sma_200).mean()
    df[["Close","7 days SMA","30 days SMA", "200 days SMA"]].iplot(title = "Graph showing Simple Moving averages of {} stock".format(ticker))
    
    
    check_buying_signal = False
    check_selling_signal = False
    for index, row in df.iterrows():
        if row["7 days SMA"] > (row["30 days SMA"] and row["200 days SMA"]):

            check_selling_signal = False
            if(check_buying_signal == False):
                check_buying_signal = True
                print("The SMA buying signal of {} stock is seen at {}".format(ticker,index))


        else:
            check_buying_signal = False

            if(check_selling_signal == False):
                check_selling_signal = True
                print("The SMA selling signal of {} stock is seen at {}".format(ticker,index))
                print("")
    return df

In [5]:
def get_rsi(ticker,start_date,end_date):
    df = yf.download(tickers = ticker, start = start_date, end = end_date)
    qf = cf.QuantFig(df)
    qf.add_rsi(periods = 20, rsi_upper=70, rsi_lower=30)
    qf.iplot()
    rsi = talib.RSI(np.array(df.Close), timeperiod=20)
    df["rsi"] = rsi
    
    check_buying_signal = False
    check_selling_signal = False
    for index, row in df.iterrows():
        if row["rsi"] < 30:

            check_selling_signal = False
            if(check_buying_signal == False):
                check_buying_signal = True
                print("The RSI buying signal of {} stock is seen at {}".format(ticker,index))


        if row["rsi"] > 70:
            check_buying_signal = False

            if(check_selling_signal == False):
                check_selling_signal = True
                print("The RSI selling signal of {} stock is seen at {}".format(ticker,index))
                print("")
    
    return df

In [6]:
def get_rsi_second(ticker,start_date,end_date):
    df = yf.download(tickers = ticker, start = start_date, end = end_date)
    qf = cf.QuantFig(df)
    qf.add_rsi(periods = 20, rsi_upper=70, rsi_lower=30)
    qf.iplot()
    
    df["U"] = np.where(df.Close.diff() > 0, df.Close.diff(), 0)
    df["D"] = np.where(df.Close.diff() < 0, -df.Close.diff(), 0)
    periods = 20
    df["MA_U"] = df.U.rolling(periods).mean()
    df["MA_D"] = df.D.rolling(periods).mean()
    df["rsi"] = df.MA_U / (df.MA_U + df.MA_D) * 100
    
    check_buying_signal = False
    check_selling_signal = False
    for index, row in df.iterrows():
        if row["rsi"] < 30:

            check_selling_signal = False
            if(check_buying_signal == False):
                check_buying_signal = True
                print("The RSI buying signal of {} stock is seen at {}".format(ticker,index))


        if row["rsi"] > 70:
            check_buying_signal = False

            if(check_selling_signal == False):
                check_selling_signal = True
                print("The RSI selling signal of {} stock is seen at {}".format(ticker,index))
                print("")
    
    return df

In [7]:
def get_daily_results(ticker,start_date,end_date):
    results_df = {}
    df = yf.download(tickers = ticker, start = start_date, end = end_date)
    sma_7 = 7
    sma_30 = 30
    sma_200 = 200
    df["7 days SMA"] = df.Close.rolling(sma_7).mean()
    df["30 days SMA"] = df.Close.rolling(sma_30).mean()
    df["200 days SMA"] = df.Close.rolling(sma_200).mean()
    df[["Close","7 days SMA","30 days SMA", "200 days SMA"]].iplot(title = "Graph showing Simple Moving averages of {} stock".format(ticker))
    
    # RSI

    qf = cf.QuantFig(df)
    qf.add_rsi(periods = 20, rsi_upper=70, rsi_lower=30)
    qf.iplot()

    df["U"] = np.where(df.Close.diff() > 0, df.Close.diff(), 0)
    df["D"] = np.where(df.Close.diff() < 0, -df.Close.diff(), 0)
    periods = 20
    df["MA_U"] = df.U.rolling(periods).mean()
    df["MA_D"] = df.D.rolling(periods).mean()
    df["rsi"] = df.MA_U / (df.MA_U + df.MA_D) * 100

    last_row = df.tail(1).reset_index()

    results_df['Tickers'] = ticker
    results_df['Date'] = last_row['Date'][0].date().strftime('%Y-%m-%d')
    results_df["Price"] = last_row["Close"].values[0]
    results_df["Volume"] = last_row["Volume"].values[0]
    results_df["SMA S"] = last_row["7 days SMA"].values[0]
    results_df["SMA M"] = last_row["30 days SMA"].values[0]
    results_df["SMA L"] = last_row["200 days SMA"].values[0]
    results_df["RSI"] = last_row["rsi"].values[0]

    if results_df["SMA S"] > results_df["SMA M"] and results_df["SMA S"] > results_df["SMA L"]:
        results_df["SMA Status"] = "Buy"
    else: 
        results_df["SMA Status"] = "Do Nothing"



    if results_df["RSI"] < 30:
        results_df["RSI Status"] = "Buy"
    elif results_df["RSI"] < 70 and results_df["RSI"] > 30:
        results_df["RSI Status"] = "Do Nothing"
    else:
        results_df["RSI Status"] = "Sell" 

    results_df = pd.DataFrame(results_df, index=[1])

    return results_df


    

    

In [8]:
def get_analysis(list_tickers):
    fundamental_df = {}
    list_tickers = []
    list_sector = []
    list_pe = []
    
    for ticker in list_tickers:
        ticker_info = yf.Ticker(ticker).info
        list_tickers.append(ticker)
        list_sector.append(ticker_info['sector'])
        list_pe.append(ticker_info["trailingPE"])
        
    fundamental_df['Ticker'] = list_tickers
    fundamental_df['Sector'] = list_sector
    fundamental_df['Price to Erning Ratio'] = list_pe
    
    fundamental_df = pd.DataFrame(fundamental_df, index=None)
    
    return fundamental_df

In [9]:
def get_percentile(datafile, sheet_name="SPX Index"):
    sectors_dictionary = {}
    list_percentile = []
    list_pe = []
    list_peer_analysis_status = []
    selected_tickers_df = pd.read_excel(datafile, sheet_name=sheet_name)
    selected_tickers_df["Symbol"] = selected_tickers_df['TICKER'].str.split(' ').str[0] 
    for symbol in list(selected_tickers_df["Symbol"]):

        try:
            ticker_info = yf.Ticker(symbol).info
            pe_ratio = ticker_info['forwardPE']
            list_pe.append(pe_ratio)
                    
        except requests.exceptions.HTTPError as http_err:
            print(f"HTTP error occurred: {http_err}")
            list_pe.append(0)

            
        except KeyError as key_err:
            print(f"KeyError occurred: {key_err}")
            list_pe.append(0)

    selected_tickers_df["PE Ratio"] = list_pe
    unique_sectors = selected_tickers_df["SECTOR"].unique()
    for sector in unique_sectors:
        sector_df = selected_tickers_df.loc[selected_tickers_df["SECTOR"] == sector]
        sector_list = list(sector_df["PE Ratio"])
        sectors_dictionary[sector] = sector_list
    for symbol in list(selected_tickers_df["Symbol"]):
        symbol_df = selected_tickers_df.loc[selected_tickers_df["Symbol"] == symbol]
        symbol_pe = float(symbol_df["PE Ratio"])
        symbol_sector = list(symbol_df["SECTOR"])
        sector_pe = sectors_dictionary[symbol_sector[0]]
        percentile = stats.percentileofscore(sector_pe,symbol_pe)
        list_percentile.append(round(percentile))
        if(percentile >= 70):
            list_peer_analysis_status.append("Sell")
        elif(percentile < 70 and percentile > 30):
            list_peer_analysis_status.append("Do Nothing")
        elif(percentile <= 30):
            list_peer_analysis_status.append("Buy")
            
    selected_tickers_df['Peer Analysis'] = list_percentile
    selected_tickers_df['Peer Analysis Status'] = list_peer_analysis_status
    return selected_tickers_df

In [10]:
def get_peer_analysis(ticker,df):
    ticker_df = df.loc[df["Symbol"] == ticker]
    pe_ratio = float(ticker_df["PE Ratio"])
    peer_analysis = int(ticker_df["Peer Analysis"])
    peer_analysis_status = list(ticker_df["Peer Analysis Status"])[0]
    return pe_ratio,peer_analysis,peer_analysis_status

In [11]:
def get_all_stocks_results(tickers_list,start_date,end_date,percentile_df):
    results_df = {}
    list_tickers = []
    list_date = []
    list_price = []
    list_volume = []
    list_SMA_S = []
    list_SMA_M = []
    list_SMA_L = []
    list_RSI = []
    list_pe = []
    list_peer_analysis = []
    list_peer_analysis_status = []
    list_SMA_status = []
    list_RSI_status = []
    
    for ticker in tickers_list:
        df = yf.download(tickers = ticker, start = start_date, end = end_date)
        
        # SMA
        sma_7 = 7
        sma_30 = 30
        sma_200 = 200
        df["7 days SMA"] = df.Close.rolling(sma_7).mean()
        df["30 days SMA"] = df.Close.rolling(sma_30).mean()
        df["200 days SMA"] = df.Close.rolling(sma_200).mean()
        
        #RSI
        df["U"] = np.where(df.Close.diff() > 0, df.Close.diff(), 0)
        df["D"] = np.where(df.Close.diff() < 0, -df.Close.diff(), 0)
        periods = 20
        df["MA_U"] = df.U.rolling(periods).mean()
        df["MA_D"] = df.D.rolling(periods).mean()
        df["rsi"] = df.MA_U / (df.MA_U + df.MA_D) * 100
        
        last_row = df.tail(1).reset_index()
        
        list_tickers.append(ticker)
        list_date.append(last_row['Date'][0].date().strftime('%Y-%m-%d'))
        list_price.append(last_row["Close"].values[0])
        list_volume.append(last_row["Volume"].values[0])
        list_SMA_S.append(last_row["7 days SMA"].values[0])
        list_SMA_M.append(last_row["30 days SMA"].values[0])
        list_SMA_L.append(last_row["200 days SMA"].values[0])
        list_RSI.append(last_row["rsi"].values[0])
        
        #check SMA
        if list_SMA_S[-1] > list_SMA_M[-1] and list_SMA_S[-1] > list_SMA_L[-1]:
            list_SMA_status.append("Buy")
        else: 
            list_SMA_status.append("Do Nothing")
            
        #check RSI
        if list_RSI[-1] < 30:
            list_RSI_status.append("Buy")
        elif list_RSI[-1] < 70 and list_RSI[-1] > 30:
            list_RSI_status.append("Do Nothing")
        else:
            list_RSI_status.append("Sell") 
        
        
        #peer analysis 
        pe, peer_analysis, status = get_peer_analysis(ticker,percentile_df)
        
        list_pe.append(pe)
        list_peer_analysis.append(peer_analysis)
        list_peer_analysis_status.append(status)
    
    #make a dataframe
    results_df['Tickers'] = list_tickers
    results_df['Date'] = list_date
    results_df["Price"] = list_price
    results_df["Volume"] = list_volume
    results_df["SMA S"] = list_SMA_S
    results_df["SMA M"] = list_SMA_M
    results_df["SMA L"] = list_SMA_L
    results_df["RSI"] = list_RSI
    results_df["PE Ratio"] = list_pe
    results_df["Peer Analysis"] = list_peer_analysis
    results_df["SMA Status"]= list_SMA_status
    results_df["RSI Status"]= list_RSI_status
    results_df["Peer Analysis Status"] = list_peer_analysis_status
    
    results_df = pd.DataFrame(results_df, index=None)

    return results_df

List of tickers

In [12]:
Nasdaq = (["MELI","MDLZ","AMZN","CPRT","DDOG","GOOG",
                  "CSGP","ABNB","TEAM", "CSCO","INTC","PANW",
                   "MSFT","NVDA","CTSH","ISRG","ALGN","EBAY"])
len(Nasdaq)

18

In [13]:
NGX = (["DANGCEM","MTNN","AIRTELAF","BUACEMEN","BUAFOODS","GTCO","TRANSCOR","ZENITHBA","NESTLE","SEPLAT",
        "STANBIC","FBNH","ACCESSCO","WAPCO","UBA","OKOMUOIL","NB","PRESCO","DANGSUGA","ETI","CUSTODIA",
        "FIDELITY","GUINNESS","INTBREW","TOTAL","FLOURMIL","FCMB","UNILEVER","UCAP","WEMABANK"])
len(NGX)

30

In [14]:
SP500 = (["LYB","AXP","VZ","AVGO","BA","CAT","JPM","CVX","KO","ABBV",
          "DIS","FLT","EXR","XOM","PSX","GE","HPQ","HD","MPWR","IBM",
          "JNJ","MCD","MRK","MMM","AWK","BAC","PFE","PG","T","TRV",
          "RTX","ADI","WMT","CSCO","INTC","GM","MSFT","DG","CI","KMI",
          "C","AIG","MO","HCA","IP","HPE","ABT","AFL","APD","RCL"
])
len(SP500)

50

Analysing one stock

Getting Historical Data

In [15]:
ticker = "MSFT"
stocks_hist = get_historical_data(ticker)
stocks_hist

[*********************100%***********************]  1 of 1 completed

The Historical data of MSFT stock  starts from 1986-03-13 00:00:00
The Historical data of MSFT stock ends at 2023-08-29 00:00:00



,Open,High,Low,Close,Volume
Date,,,,,
1986-03-13,0.055004,0.063093,0.055004,0.060396,1031788800
1986-03-14,0.060396,0.063632,0.060396,0.062553,308160000
1986-03-17,0.062553,0.064172,0.062553,0.063632,133171200
1986-03-18,0.063632,0.064172,0.061475,0.062014,67766400
1986-03-19,0.062014,0.062553,0.060396,0.060936,47894400
...,...,...,...,...,...
2023-08-23,323.820007,329.200012,323.459991,327.000000,21166400
2023-08-24,332.850006,332.980011,319.959991,319.970001,23281400
2023-08-25,321.470001,325.359985,318.799988,322.980011,21671400


fundamental data

In [16]:
stock_fundamental = get_fundamental_data(ticker)
stock_fundamental

Key fundamental data about MSFT stock
The Price-to-Earnings ratio of MSFT stock is : 33.75645
The price-to-Book value of MSFT stock is : 11.788238
The Enterprise Value-to-EBITDA of MSFT stock is : 23.209


,address1,city,state,zip,country,phone,website,industry,industryDisp,sector,...,returnOnEquity,freeCashflow,operatingCashflow,earningsGrowth,revenueGrowth,grossMargins,ebitdaMargins,operatingMargins,financialCurrency,trailingPegRatio
0,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
1,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
2,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
3,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
4,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
5,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
6,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
7,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
8,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699
9,One Microsoft Way,Redmond,WA,98052-6399,United States,425 882 8080,https://www.microsoft.com,Software—Infrastructure,Software—Infrastructure,Technology,...,0.38824,47268999168,87581999104,0.202,0.083,0.6892,0.48143,0.41773,USD,2.3699


Getting SMA and RSI

In [17]:
today = pd.Timestamp.now().floor('D')
today 

Timestamp('2023-08-29 00:00:00')

In [18]:
start = "2022-01-01"
end = today
mtn_sma = get_sma(ticker,start,end)

[*********************100%***********************]  1 of 1 completed


The SMA selling signal of MSFT stock is seen at 2022-01-03 00:00:00

The SMA buying signal of MSFT stock is seen at 2023-02-07 00:00:00
The SMA selling signal of MSFT stock is seen at 2023-02-28 00:00:00

The SMA buying signal of MSFT stock is seen at 2023-03-09 00:00:00


In [19]:
mtn_rsi = get_rsi(ticker,start,end)

[*********************100%***********************]  1 of 1 completed


The RSI selling signal of MSFT stock is seen at 2023-05-26 00:00:00



In [20]:
mtn_rsi_second = get_rsi_second(ticker,start,end)

[*********************100%***********************]  1 of 1 completed


The RSI selling signal of MSFT stock is seen at 2022-08-10 00:00:00

The RSI buying signal of MSFT stock is seen at 2022-09-02 00:00:00
The RSI selling signal of MSFT stock is seen at 2022-12-01 00:00:00

The RSI buying signal of MSFT stock is seen at 2023-08-15 00:00:00


In [21]:
last_day_data= get_daily_results(ticker,start,end)


[*********************100%***********************]  1 of 1 completed


In [22]:
last_day_data


,Tickers,Date,Price,Volume,SMA S,SMA M,SMA L,RSI,SMA Status,RSI Status
1,MSFT,2023-08-28,323.700012,14802100,322.067147,330.744666,286.79485,39.073654,Do Nothing,Do Nothing


Analysing Many stocks

In [23]:
tickers = ["AAPL","MSFT","TSLA"]
for ticker in tickers:
    print("Information about {} stock".format(ticker))
    print("==========================================")
    print("")
    historical_data = get_historical_data(ticker)
    fundamental_data = get_fundamental_data(ticker)
    
    sma_signal = get_sma(ticker,start,end)
    rsi_signal = get_rsi_second(ticker,start,end)
    

Information about AAPL stock

[*********************100%***********************]  1 of 1 completed

The Historical data of AAPL stock  starts from 1980-12-12 00:00:00
The Historical data of AAPL stock ends at 2023-08-29 00:00:00

Key fundamental data about AAPL stock
The Price-to-Earnings ratio of AAPL stock is : 30.610739
The price-to-Book value of AAPL stock is : 47.36241
The Enterprise Value-to-EBITDA of AAPL stock is : 23.104
[*********************100%***********************]  1 of 1 completed


The SMA selling signal of AAPL stock is seen at 2022-01-03 00:00:00

The SMA buying signal of AAPL stock is seen at 2023-02-06 00:00:00
[*********************100%***********************]  1 of 1 completed


The RSI buying signal of AAPL stock is seen at 2022-04-27 00:00:00
The RSI selling signal of AAPL stock is seen at 2022-07-19 00:00:00

The RSI buying signal of AAPL stock is seen at 2022-09-15 00:00:00
The RSI selling signal of AAPL stock is seen at 2023-01-27 00:00:00

The RSI buying signal of AAPL stock is seen at 2023-08-14 00:00:00
Information about MSFT stock

[*********************100%***********************]  1 of 1 completed

The Historical data of MSFT stock  starts from 1986-03-13 00:00:00
The Historical data of MSFT stock ends at 2023-08-29 00:00:00

Key fundamental data about MSFT stock
The Price-to-Earnings ratio of MSFT stock is : 33.768837
The price-to-Book value of MSFT stock is : 11.7925625
The Enterprise Value-to-EBITDA of MSFT stock is : 23.209
[*********************100%***********************]  1 of 1 completed


The SMA selling signal of MSFT stock is seen at 2022-01-03 00:00:00

The SMA buying signal of MSFT stock is seen at 2023-02-07 00:00:00
The SMA selling signal of MSFT stock is seen at 2023-02-28 00:00:00

The SMA buying signal of MSFT stock is seen at 2023-03-09 00:00:00
[*********************100%***********************]  1 of 1 completed


The RSI selling signal of MSFT stock is seen at 2022-08-10 00:00:00

The RSI buying signal of MSFT stock is seen at 2022-09-02 00:00:00
The RSI selling signal of MSFT stock is seen at 2022-12-01 00:00:00

The RSI buying signal of MSFT stock is seen at 2023-08-15 00:00:00
Information about TSLA stock

[*********************100%***********************]  1 of 1 completed

The Historical data of TSLA stock  starts from 2010-06-29 00:00:00
The Historical data of TSLA stock ends at 2023-08-29 00:00:00

Key fundamental data about TSLA stock
The Price-to-Earnings ratio of TSLA stock is : 68.46027
The price-to-Book value of TSLA stock is : 15.5118265
The Enterprise Value-to-EBITDA of TSLA stock is : 44.119
[*********************100%***********************]  1 of 1 completed


The SMA selling signal of TSLA stock is seen at 2022-01-03 00:00:00

The SMA buying signal of TSLA stock is seen at 2023-06-05 00:00:00
[*********************100%***********************]  1 of 1 completed


The RSI selling signal of TSLA stock is seen at 2022-03-23 00:00:00

The RSI buying signal of TSLA stock is seen at 2022-05-03 00:00:00
The RSI selling signal of TSLA stock is seen at 2022-07-28 00:00:00

The RSI buying signal of TSLA stock is seen at 2022-10-07 00:00:00
The RSI selling signal of TSLA stock is seen at 2023-01-26 00:00:00

The RSI buying signal of TSLA stock is seen at 2023-05-01 00:00:00
The RSI selling signal of TSLA stock is seen at 2023-05-24 00:00:00

The RSI buying signal of TSLA stock is seen at 2023-08-14 00:00:00


In [24]:
tickers_percentile = get_percentile("List of Tickers.xlsx")

In [25]:
sp_index = get_all_stocks_results(tickers_percentile["Symbol"],start,end,tickers_percentile)

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%********

In [26]:
sp_index

,Tickers,Date,Price,Volume,SMA S,SMA M,SMA L,RSI,PE Ratio,Peer Analysis,SMA Status,RSI Status,Peer Analysis Status
0,AAPL,2023-08-28,180.190002,43769700,177.694286,184.852001,162.559850,31.250007,27.605143,79,Do Nothing,Do Nothing,Sell
1,MSFT,2023-08-28,323.700012,14802100,322.067147,330.744666,286.794850,39.073654,25.937798,71,Do Nothing,Do Nothing,Sell
2,GOOGL,2023-08-28,131.009995,20526900,129.707141,128.290999,107.637250,46.834491,20.124437,80,Buy,Do Nothing,Sell
3,GOOG,2023-08-28,131.789993,16706400,130.405714,128.789333,108.197775,47.596498,20.417550,90,Buy,Do Nothing,Sell
4,AMZN,2023-08-28,133.139999,34083100,133.701427,133.897668,108.325400,49.366204,42.792890,88,Do Nothing,Do Nothing,Sell
...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,SLB,2023-08-28,57.830002,4811300,57.264285,57.744667,51.923050,48.021729,15.736339,100,Do Nothing,Do Nothing,Sell
96,ETN,2023-08-28,226.240005,1611300,220.054286,214.642333,177.253150,72.968820,23.329742,80,Buy,Sell,Sell
97,PYPL,2023-08-28,61.669998,10447700,60.748570,66.413000,72.231025,27.287313,11.067376,4,Do Nothing,Buy,Buy
98,MO,2023-08-28,43.970001,5069200,43.224286,44.260000,45.448700,35.756426,8.437500,12,Do Nothing,Do Nothing,Buy


In [27]:
file_name = 'Stocks_results_'+ str(today).split()[0] +'.csv'
sp_index.to_csv(file_name, index=False)


In [28]:
olApp = win32.Dispatch('Outlook.Application')
olNS = olApp.GetNameSpace('MAPI')

# construct the email item object
mailItem = olApp.CreateItem(0)
mailItem.Subject = 'Trading Analytics- Stocks Results on {}'.format(today)
mailItem.BodyFormat = 1
mailItem.Body = """Dear Investment Team,

The attached document shows the results of 100 stocks. It shows the SMA, RSI and Peer Analysis statuses.
"""
mailItem.To = 'Nathaniel.Akinwunmi@arm.com.ng'

cc_recipients = ['Ghislain.Bisamaza@arm.com.ng','Ayodele.Oluleye@arm.com.ng']
mailItem.CC = ";".join(cc_recipients)


mailItem.Attachments.Add(os.path.join(os.getcwd(), file_name))
mailItem.Display()
# mailItem.Send()